# Программа парсер на Python
## Цели
* Скачать access.log
* Сформировать CSV файл со списком уникальных IP-адресов из get-запросов. Для каждого из них должно быть указано успешных get-запросов с этого адреса, и количество всех остальных get-запросов.
* Вывести на экран список TOP30 IP адресов, с наибольшим количеством успешных get-запросов.

# Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np
import wget
import re

import urllib
from apachelogs import LogParser

Сформируем лист logs. После чего удалим из него пустые строки.

In [2]:
url = 'http://www.almhuette-raith.at/apache-log/access.log'

In [3]:
with urllib.request.urlopen(url) as fp:
    logs = []
    [logs.append(fp.readline().decode("utf-8")) for _ in range(20000)]
logs.remove('\n')

Так же файл целиком можно загрузить через wget. А потом прочитать нужное количество данных из файла.

    wget.download('http://www.almhuette-raith.at/apache-log/access.log')
    
    with open('access.log') as fp:
        logs = []
        [logs.append(fp.readline()) for _ in range(20000)]
    logs.remove('\n')

In [4]:
logs[:10]

['13.66.139.0 - - [19/Dec/2020:13:57:26 +0100] "GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1" 200 32653 "-" "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)" "-"\n',
 '157.48.153.185 - - [19/Dec/2020:14:08:06 +0100] "GET /apache-log/access.log HTTP/1.1" 200 233 "-" "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" "-"\n',
 '157.48.153.185 - - [19/Dec/2020:14:08:08 +0100] "GET /favicon.ico HTTP/1.1" 404 217 "http://www.almhuette-raith.at/apache-log/access.log" "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" "-"\n',
 '216.244.66.230 - - [19/Dec/2020:14:14:26 +0100] "GET /robots.txt HTTP/1.1" 200 304 "-" "Mozilla/5.0 (compatible; DotBot/1.1; http://www.opensiteexplorer.org/dotbot, help@moz.com)" "-"\n',
 '54.36.148.92 - - [19/Dec/2020:14:16:44 +0100] "GET /index.php?option=com_phocagall

* Данные имеют не стандартный отличный от "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"" формат из-за того что на конце  "-"\n
* Применим как apachelogs.LogParser так и регулярные выражения.

# Формирование CSV файл со списком уникальных IP-адресов из get-запросов

## Используем apachelogs.LogParser

Так как логи содержат строки с элементами которые мешают их автоматической обработке, то будем их сокращать на 6 символов перед отправкой в парсер.

In [5]:
dic = {}; i=0
parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")
for i in range(len(logs)):
    # Парсим данные.
    entry = parser.parse(logs[i][0:-5])
    # Заносим нужную информацию в словарь.
    dic[i] = [entry.remote_host, entry.request_line.split()[0], entry.final_status]

Получаем датафрейм который можно сохранить в .csv

In [6]:
df_apache = pd.DataFrame.from_dict(dic, orient='index', columns=['ip','request','status'])
# Оставляем только GET запросы
df_apache = df_apache[df_apache.request == 'GET']
df_apache.head(10)

,ip,request,status
0,13.66.139.0,GET,200
1,157.48.153.185,GET,200
2,157.48.153.185,GET,404
3,216.244.66.230,GET,200
4,54.36.148.92,GET,200
5,92.101.35.224,GET,200
6,73.166.162.225,GET,200
7,73.166.162.225,GET,404
8,54.36.148.108,GET,200
9,54.36.148.1,GET,200


## Используем регулярные выражения

Сформируем датафрейм из трех столбцов: IP-адреса, запросы, из кодов запросов.

Распакуем не пустые списки, присвоив остальным значения NaN чтобы удалить строки их содержащие (останутся только запросы GET).

In [7]:
df_re = pd.DataFrame(
    {
        'ip': list(map(lambda x: re.findall(r'(?:\d{1,3}[.]){3}(?:\d{1,3})', x), logs)),
        'request': list(map(lambda x: re.findall(r'GET', x), logs)),
        'status': list(map(lambda x: re.findall(r'\s(\d\d\d)\s\d', x), logs)),
    }
)

# Распаковываем массивы 
df_re = df_re.applymap(lambda x: x[0] if x != [] else np.nan).dropna(axis=0, how='any')

# Поменяем тип на int
df_re.status = df_re.status.map(int)

df_re.head(10)

,ip,request,status
0,13.66.139.0,GET,200
1,157.48.153.185,GET,200
2,157.48.153.185,GET,404
3,216.244.66.230,GET,200
4,54.36.148.92,GET,200
5,92.101.35.224,GET,200
6,73.166.162.225,GET,200
7,73.166.162.225,GET,404
8,54.36.148.108,GET,200
9,54.36.148.1,GET,200


## Сверка полученных результатов

In [8]:
df_re.equals(df_apache)

True

Видно, что датафреймы идентичны, идем дальше.

In [9]:
df = df_apache

# TOP30 IP адресов, с наибольшим количеством успешных get-запросов

Определим новый столбец, выделив является ли запрос успешным (2**) или нет.

In [10]:
df['status_2**'] = df.status.apply(lambda x: 200<=x<300)

Сгруппируем данные по ip.

In [11]:
pivot = df.pivot_table(index='ip', columns='status_2**', values='request', aggfunc='count') \
             .reset_index()
# Дадим столбцам адекватные имена.
pivot.columns = ['ip','other_req','success_req']

In [12]:
print('Уникальных IP:',pivot.shape[0])
pivot.sort_values('success_req', ascending=False).head(30)

Уникальных IP: 482


,ip,other_req,success_req
314,45.144.0.179,NaN,473.0
136,176.222.58.254,NaN,467.0
299,45.132.207.154,NaN,445.0
320,45.153.227.55,NaN,444.0
309,45.138.4.22,NaN,440.0
319,45.153.227.31,NaN,438.0
310,45.138.4.35,NaN,436.0
302,45.132.51.62,NaN,434.0
137,176.222.58.90,NaN,430.0
308,45.138.145.131,NaN,428.0


In [13]:
pivot.to_csv('the_name_you_like.csv')